In [2]:
import os
import logging
import requests
from bs4 import BeautifulSoup
import logging
from linkedin_jobs_scraper import LinkedinScraper
from linkedin_jobs_scraper.events import Events, EventData, EventMetrics
from linkedin_jobs_scraper.query import Query, QueryOptions, QueryFilters
from linkedin_jobs_scraper.filters import RelevanceFilters, TimeFilters, TypeFilters, ExperienceLevelFilters, \
    OnSiteOrRemoteFilters, SalaryBaseFilters
import pandas as pd
from linkedin_jobs_scraper.utils.chrome_driver import build_driver
import time
from datetime import datetime
import csv

In [3]:
FieldName = ['job_id', 'title', 'company','date','link','insight','description']
today = datetime.now().strftime("%y_%m_%d")
target_csv = f"job_scan_{today}.csv"

# Create the output csv if not exist
if target_csv and not os.path.exists(target_csv):
    with open(target_csv, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file,fieldnames=FieldName)
        writer.writeheader()


In [4]:
# Function to append each result to the CSV file
def write_output(result, csv_file,FieldName = FieldName):
    with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=FieldName)
        writer.writerow(result)


# Change root logger level (default is WARN)
logging.basicConfig(level=logging.INFO)


# Fired once for each successfully processed job
def on_data(data: EventData):
    print(target_csv)
    print('[ON_DATA]',data.job_id, data.title, data.company, data.date, data.link, data.insights,
          len(data.description))
    output = {'job_id':data.job_id, 'title':data.title, 'company':data.company,'date':data.date,'link':data.link,'insight':data.insights,'description':data.description}
    write_output(output, target_csv)

# Fired once for each page (25 jobs)
def on_metrics(metrics: EventMetrics):
    print('[ON_METRICS]', str(metrics))


def on_error(error):
    print('[ON_ERROR]', error)


def on_end():
    print('[ON_END]')


scraper = LinkedinScraper(
    chrome_executable_path=None,  # Custom Chrome executable path (e.g. /foo/bar/bin/chromedriver)
    chrome_binary_location=None,  
    chrome_options=None,  # Custom Chrome options here
    headless=True,  # Overrides headless mode only if chrome_options is None
    max_workers=1,  # How many threads will be spawned to run queries concurrently (one Chrome driver for each thread)
    slow_mo=0.5,  # Slow down the scraper to avoid 'Too many requests 429' errors (in seconds)
    page_load_timeout=40,
  # Page load timeout (in seconds)    
)

# Add event listeners
scraper.on(Events.DATA, on_data)
scraper.on(Events.ERROR, on_error)
scraper.on(Events.END, on_end)


INFO:li:scraper:('Using strategy AuthenticatedStrategy',)


In [5]:
linked_page = pd.read_csv('linkedin_pages.csv')
execute_list = linked_page.dropna()
for link in execute_list['job_link']:
    query = Query(    
        options=QueryOptions(
            locations=['United States'],
            skip_promoted_jobs=True,
            limit = 1000,      
            filters=QueryFilters(
                company_jobs_url= link
            )
        )
    )
    scraper.run(query)

INFO:li:scraper:('Starting new query', "Query(query= options=QueryOptions(limit=1000 locations=['United States'] filters=QueryFilters(company_jobs_url=https://www.linkedin.com/jobs/search/?f_C=6640471&geoId=92000000&origin=COMPANY_PAGE_JOBS_CLUSTER_EXPANSION&originToLandingJobPostings=3971570718%2C3973575366) apply_link=False skip_promoted_jobs=True page_offset=0))")
INFO:li:scraper:('Chrome debugger url', 'http://localhost:52662')
INFO:li:scraper:('Websocket debugger url: ', 'ws://localhost:52662/devtools/page/B4806963BEA75228F6CEA2B81D986532')
INFO:li:scraper:('[][United States]', 'Setting authentication cookie')
INFO:li:scraper:('[][United States]', 'Opening https://www.linkedin.com/jobs/search?location=United+States&f_C=6640471&start=0')
INFO:li:scraper:('[][United States]', 'Session is valid')
INFO:li:scraper:('[][United States][1]', 'Processed')


job_scan_24_07_18.csv
[ON_DATA] 3971570718 Senior Data Analyst with verification Solera Health  https://www.linkedin.com/jobs/view/3971570718/?eBP=NON_CHARGEABLE_CHANNEL&refId=Ec2rSjDYSFjQ%2Bv71OLh7vw%3D%3D&trackingId=WJ6Tn50EEj1ijUQuTUgGhQ%3D%3D&trk=flagship3_search_srp_jobs ['Remote Full-time Mid-Senior level', '51-200 employees · Hospitals and Health Care', 'Skills: Data Analytics, SQL, +8 more', 'See how you compare to over 100 other applicants. Try Premium for $0', '', 'Am I a good fit for this job?', 'How can I best position myself for this job?', 'Tell me more about Solera He...'] 5241


INFO:li:scraper:('[][United States][2]', 'Processed')


job_scan_24_07_18.csv
[ON_DATA] 3973575366 Compliance Manager with verification Solera Health 2024-07-11 https://www.linkedin.com/jobs/view/3973575366/?eBP=NON_CHARGEABLE_CHANNEL&refId=Ec2rSjDYSFjQ%2Bv71OLh7vw%3D%3D&trackingId=Nyk6xrxMedXXWDsZodzdPQ%3D%3D&trk=flagship3_search_srp_jobs ['Remote Full-time Mid-Senior level', '51-200 employees · Hospitals and Health Care', 'Skills: Healthcare Compliance, External Audits, +8 more', 'See how you compare to over 100 other applicants. Try Premium for $0', '', 'Am I a good fit for this job?', 'How can I best position myself for this job?', 'Tell me more about Solera He...'] 4870


INFO:li:scraper:('[][United States][3]', 'Skipped because promoted')
INFO:li:scraper:('[][United States][4]', 'Skipped because promoted')
INFO:li:scraper:('[][United States][5]', 'Skipped because promoted')
INFO:li:scraper:('[][United States]', 'No more jobs to process in this page')
INFO:li:scraper:('[][United States]', 'Metrics:', '{ processed: 2, failed: 0, missed: 20, skipped: 3 }')
INFO:li:scraper:('[][United States]', 'Pagination requested [1]')
INFO:li:scraper:('[][United States]', 'Opening https://www.linkedin.com/jobs/search?location=United+States&f_C=6640471&start=25')
INFO:li:scraper:('[][United States]', 'Waiting for new jobs to load')
INFO:li:scraper:('[][United States]', "Couldn't find more jobs for the running query")


...

INFO:li:scraper:('[][United States][79]', 'Processed')


job_scan_24_07_18.csv
[ON_DATA] 3964765242 Clinical Veterinarian with verification Altasciences 2024-07-01 https://www.linkedin.com/jobs/view/3964765242/?eBP=NON_CHARGEABLE_CHANNEL&refId=2QQXIb6UzJgyME%2B78sJZTQ%3D%3D&trackingId=5R0bpRWRr9CJmY4ENbhs6Q%3D%3D&trk=flagship3_search_srp_jobs ['Full-time Entry level', '1,001-5,000 employees · Research Services', 'Skills: Veterinary Medicine, Computer Literacy, +8 more', 'See recent hiring trends for Altasciences. Try Premium for $0', '', 'Am I a good fit for this job?', 'How can I best position myself for this job?', 'Tell me more about Altasciences'] 5009


INFO:li:scraper:('[][United States][80]', 'Processed')


job_scan_24_07_18.csv
[ON_DATA] 3963246090 Senior Project Finance Analyst Lead with verification Altasciences 2024-06-29 https://www.linkedin.com/jobs/view/3963246090/?eBP=NON_CHARGEABLE_CHANNEL&refId=2QQXIb6UzJgyME%2B78sJZTQ%3D%3D&trackingId=ka6NJiut10Xu94YUrfy9Xg%3D%3D&trk=flagship3_search_srp_jobs ['$73K/yr - $98K/yr Full-time Mid-Senior level', '1,001-5,000 employees · Research Services', 'Skills: Deferred Revenue, Reporting Requirements, +8 more', 'See how you compare to 3 applicants. Try Premium for $0', '', 'Am I a good fit for this job?', 'How can I best position myself for this job?', 'Tell me more about Altasciences'] 7694


INFO:li:scraper:('[][United States][81]', 'Processed')
